In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.signal import TransferFunction, bode, lsim, lti
from scipy import signal
from scipy.integrate import solve_ivp
from IPython.display import display, Latex, Markdown, Image, Math

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

In [ ]:
class ReglerPid:
    def __init__(self, setPoint, P, I, D, outMin, outMax, sampleTime, aw=True):
        self.P = P
        self.I = I
        self.D = D
        self.setPoint = setPoint
        self.outMin = outMin
        self.outMax = outMax
        self.sampleTime = sampleTime
        self.e = 0
        self.int = 0
        self.aw = aw
        self.outWindup = 0

    def reset(self):
        self.e = 0
        self.int = 0
    
    def regel(self, curValue):
        e = self.setPoint - curValue

        intStep = (e + self.e) * self.sampleTime
        diffStep = (e - self.e) / self.sampleTime

        self.outWindup = (self.P * e +
                          self.I * (self.int + intStep) / 2.0 +
                          self.D * diffStep)

        if self.outWindup < self.outMin:
            out = self.outMin
        elif self.outWindup > self.outMax:
            out = self.outMax
        else:
            out = self.outWindup
            if self.aw:
                self.int += intStep

        if not self.aw:
            self.int += intStep
            
        self.e = e

        return out

## PID-Regler mit Beschränkungen

In [ ]:
imag = Image("../../images/twoTank.png", width=800)
display(imag)

## 2-Tanksystem

- nichtlineares Modell im Zeitbereich
    \begin{align*}
        \dot{z}_1(t) & = a_{1} \sqrt{z_1(t)} + \frac{K_{u}}{A_{T,1}} u_{\text{A}}(t)\\
        \dot{z}_2(t) & = \frac{A_{T,1}}{A_{T,2}} a_{1} \sqrt{z_1(t)} + a_{2} \sqrt{z_2(t)} 
    \end{align*}
    mit Ausgang $y(t) = z_2(t)$ und
    \begin{align*}
        a_1 & = A_{S,1} \sqrt{\frac{2\, g}{A_{T,1}^2 - A_{S,1}^2}}, & 
        a_2 & = A_{S,2} \sqrt{\frac{2\, g}{A_{T,2}^2 - A_{S,2}^2}}
    \end{align*}
- PID-Regler
    \begin{align*}
        u(t) & = K_{\text{P}} e(t) + K_{\text{I}} \int_0^t e(\tau) \text{d} \tau + K_{\text{D}} \dot{e}(t)
    \end{align*}

__Definition Parammeter__

In [ ]:
tSim = np.linspace(0, 500, 501)

## Simulation

In [ ]:
def prozess(t, x, uA):
    z1 = x[0]
    z2 = x[1]

    uA0 = 6.4
    if uA < uA0:
        u = 0
    else:
        u = uA - uA0
        
    if z1 <= 1e-6:
        z1 = 0

    if z2 <= 1e-6:
        z2 = 0
        
    dx = np.zeros(2)
    dx[0] = 0.00216625315586689 * u - 0.0127646468529449 * np.sqrt(2) * np.sqrt(z1)
    dx[1] = 0.0127646468529449 * np.sqrt(2) * np.sqrt(z1) - 0.00908683019582126 * np.sqrt(2) * np.sqrt(z2)

    return dx

In [ ]:
output = widgets.Output()

with output:
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 6), sharex=True, gridspec_kw={'height_ratios': [2, 1, 1]})

plt.subplots_adjust(wspace=0.2, hspace=0.07)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.89)

ax1.set_xlim([0, tSim[-1]]) 
ax2.set_xlim([0, tSim[-1]])
ax3.set_xlim([0, tSim[-1]])
ax1.set_ylim([-0.01, 0.40])
ax2.set_ylim([-20, 40]) 
ax3.set_ylim([-0.5, 0.5]) 
ax1.grid() 
ax2.grid()
ax3.grid()
ax1.set_ylabel(r"$\tilde{z}_{\bullet}$ in m")
ax2.set_ylabel(r"$\tilde{u}_{\mathrm{A}}$ in V")
ax3.set_ylabel(r"$e$ in m")
ax3.set_xlabel(r"$t$ in s")

ax21 = ax2.twinx()
ax21.set_ylabel(r"$\tilde{u}_{Int}$ in V", color='C4')
ax21.tick_params(axis='y', colors='C4')
ax21.set_ylim([-5, 25]) 

ySys, = ax1.plot([], [], label=r'Höhe $\tilde{z}_{2}$')
yRefSys, = ax1.plot([], [], 'C1', label=r'Referenz $y_\mathrm{r} = \tilde{z}_{2,r}$')
uSys, = ax2.plot([], [], label=r'Regler $u_{\mathrm{r}}$')
iSys, = ax21.plot([], [], 'C4', label=r'Integrator')
eSys, = ax3.plot([], [], label=r'Fehler')

handlesAx, labelsAx = ax1.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.13, 0.94, 0.7675, .15), loc=3,
           ncol=3, mode="expand", borderaxespad=0., framealpha=0.5)

sliderYr = widgets.FloatSlider(value=0.2,
                               min=0,
                               max=0.25,
                               step=0.05,
                               description=r'$\tilde{z}_{2,\mathrm{r}}$')

checkAW = widgets.Checkbox(value=False,
                           description='Anti-Windup',
                           disabled=False)

def calcSystem(_):
    yR = sliderYr.value
    
    regler = ReglerPid(yR, 100, 4, 0, 0, 12, 1, checkAW.value)
        
    x0 = np.array([0, 0])
    yRes = [0]
    uRes = []
    eRes = []
    iRes = []
    regler.reset()
    for idx, x in enumerate(tSim[1:]):
        u = regler.regel(yRes[-1])
        res = solve_ivp(prozess,
                    [tSim[idx], tSim[idx+1]],
                    x0,
                    args=(u,))
        x0 = res.y.T[-1, :]
        y = res.y.T[-1, 1]
        if y >= 0.3:
            y = 0.3
        elif y <= 0.0:
            y = 0.0
        yRes.append(y)
        uRes.append(u)
        iRes.append(regler.int)
        eRes.append(regler.e)

    uSys.set_data(tSim[:-1], uRes)
    ySys.set_data(tSim, yRes)
    iSys.set_data(tSim[:-1], iRes)
    eSys.set_data(tSim[:-1], eRes)
    yRefSys.set_data(tSim[[0, -1]], np.array([yR, yR]))

    fig.canvas.draw()

sliderYr.observe(calcSystem, names='value')
checkAW.observe(calcSystem, names='value')

calcSystem(_)

contControls = HBox([sliderYr, checkAW])
contControls.layout = make_box_layout()

controls = HBox([contControls])
VBox([controls, output], layout=Layout(display='flex', flex_flow='row', justify_content='center', align_items='center'))